<a href="https://colab.research.google.com/github/NickSlm/ml1/blob/main/LSTM_Encoder_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from zipfile import ZipFile
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd
assert tf.__version__ >= "2.0"

2023-11-16 10:23:21.419045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 10:23:22.044035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nick/miniconda3/envs/tf_env/lib/python3.9/site-packages/nvidia/cudnn/lib:/home/nick/miniconda3/envs/tf_env/lib/
2023-11-16 10:23:22.044132: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nick

In [2]:
dataset_dir = os.path.join("datasets", "sentences")
file_path = os.path.join(dataset_dir, "heb-eng.zip")
with ZipFile(file_path, "r") as z:
    z.extractall(path=dataset_dir)
path_to_file = os.path.join(dataset_dir, "heb.txt")

In [6]:
with open(path_to_file, encoding="UTF-8") as file:
    lines = file.readline().strip().split('\n')
    word_pairs = [[w for w in l.split('\t')]  for l in lines]
    return word_pairs[0][:2]

['Go.\tלך!\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9435252 (Alkrasnov)']


In [4]:
vocab_size = 100
embed_size = 10

In [5]:
encoder_input = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_input = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embedding = keras.layers.Embedding(vocab_size, embed_size)
encoder_embedding = embedding(encoder_input)
decoder_embedding = embedding(decoder_input)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_output, hidden_state, cell_state = encoder(encoder_embedding)
encoder_state = [hidden_state, cell_state]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)

decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)

final_outputs, final_state, final_sequence_lengths  = decoder(decoder_embedding, initial_state=encoder_state, sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)
model = keras.Model(inputs=[encoder_input, decoder_input, sequence_lengths],outputs=[Y_proba])

2023-11-16 10:23:23.644237: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 10:23:23.869299: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 10:23:23.869377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 10:23:23.870460: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
